In [1]:
import sys
sys.path.insert(0, "../../code")

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
from datasets import *

In [4]:
d = Datasets('../../data')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (10,11,12,13,14,16,17,20,21,22,23,24,25,27,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [5]:
ssi_df = d.ssi[[
    # BDP ID
    'RECORD KEY BDP OWNER',
    'RECORD KEY BDP ACCOUNT HOLDING INSTITUTION',
    # EDGE PAYLOAD
    'ISO CURRENCY CODE',
    'ASSET CATEGORY',
    'ACCOUNT NBR WITH ACCOUNT HOLDING INSTITUTION',
]]

In [6]:
bdp_df = d.bdp[[
    # BDP ID
    'RECORD KEY',
    # NODE PAYLOAD
    'INSTITUTION NAME',
    'CITY',
    'COUNTRY NAME',
    'ISO COUNTRY CODE',
    'TIMEZONE',
    # ORG HIERARCHY
    'PARENT OFFICE KEY',
    'HEAD OFFICE KEY',
    'LEGAL PARENT KEY',
    'GROUP PARENT KEY',
    # BIC IDs
    'BIC8',
    'BRANCH BIC',
    'BIC',
    'CONNECTED BIC',
    # MORE IDs
    'OFFICE TYPE',
    'LEGAL TYPE',
    'GROUP TYPE',
    'INSTITUTION STATUS',
    'SSI GROUP KEY',
]]

In [7]:
min_bdp_df = bdp_df[[
    # BDP ID
    'RECORD KEY',
    # NODE PAYLOAD
    'ISO COUNTRY CODE',
    'TIMEZONE',
]]

In [8]:
min_edge_df = ssi_df.merge(
    min_bdp_df,
    left_on = 'RECORD KEY BDP OWNER',
    right_on = 'RECORD KEY',
).merge(
    min_bdp_df,
    left_on = 'RECORD KEY BDP ACCOUNT HOLDING INSTITUTION',
    right_on = 'RECORD KEY',
    suffixes = (' of OWNER', ' of HOLDER'),
).drop(columns = [
    'RECORD KEY BDP OWNER',
    'RECORD KEY BDP ACCOUNT HOLDING INSTITUTION',
])

In [9]:
min_edge_df.head().T

,0,1,2,3,4
ISO CURRENCY CODE,AED,AED,CAD,AED,AED
ASSET CATEGORY,ANYY,ANYY,ANYY,ANYY,ANYY
ACCOUNT NBR WITH ACCOUNT HOLDING INSTITUTION,NaN,AE430330000010195511174,AE150330000019030000078,AE270330000010195510201,AE060330000010195511161
RECORD KEY of OWNER,BD0000000B8A,BD00000005G6,BD00000005G6,BD00000006O3,BD0000000HUQ
ISO COUNTRY CODE of OWNER,SD,BH,BH,LB,OM
TIMEZONE of OWNER,K,B,B,B,R
RECORD KEY of HOLDER,BD000000ENI3,BD000000ENI3,BD000000ENI3,BD000000ENI3,BD000000ENI3
ISO COUNTRY CODE of HOLDER,AE,AE,AE,AE,AE
TIMEZONE of HOLDER,A,A,A,A,A


In [10]:
min_edge_df.equals(d.edges)

True

In [11]:
d.edges.equals(min_edge_df)

True

In [17]:
raw_ssi_nx = nx.from_pandas_edgelist(
    d.edges,
    source = 'RECORD KEY of OWNER',
    target = 'RECORD KEY of HOLDER',
    edge_attr = True,
)
connected_components = nx.connected_components(raw_ssi_nx)
first_connected_component_as_list_of_tuples = list(zip(range(1), connected_components))
first_connected_component = first_connected_component_as_list_of_tuples[0][1]
ssi_nx = nx.subgraph(raw_ssi_nx, first_connected_component)

In [18]:
print(nx.info(ssi_nx))

Name: 
Type: Graph
Number of nodes: 17706
Number of edges: 167445
Average degree:  18.9139


In [19]:
print(nx.info(d.ssi_nx))

Name: 
Type: Graph
Number of nodes: 17706
Number of edges: 167445
Average degree:  18.9139


In [24]:
len(list(nx.all_simple_paths(
    d.ssi_nx, 
    'BD0000000ZJX', 
    'BD000000VPX5', 
    4
)))

14733

In [16]:
.assign(
    cross_border   = lambda x: x['ISO COUNTRY CODE of OWNER'] != x['ISO COUNTRY CODE of HOLDER'],
    cross_timezone = lambda x: x['TIMEZONE of OWNER'] != x['TIMEZONE of HOLDER'],
)

SyntaxError: invalid syntax (<ipython-input-16-7351433e2ee5>, line 1)

In [ ]:
d.edges.sample(10)

In [26]:
d.bdp.head().T

,0,1,2,3,4
MODIFICATION FLAG,A,A,A,A,A
RECORD KEY,BD000000DAMC,BD000000DBB1,BD000000DBES,BD000000I8PJ,BD000000I8Q5
OFFICE TYPE,FB,HO,FB,FB,SF
PARENT OFFICE KEY,BD000000DAMD,BD000000DBB1,BD000000DBET,BD000000DBH8,BD000000I8PJ
HEAD OFFICE KEY,BD000000DAMD,BD000000DBB1,BD000000DBET,BD000000DBH8,BD000000DBH8
LEGAL TYPE,B,L,B,B,B
LEGAL PARENT KEY,BD000000DAMD,BD000000DBB1,BD000000DBET,BD000000DBH8,BD000000I8PJ
GROUP TYPE,Member,Member,Member,Member,Member
GROUP PARENT KEY,BD000000DAMD,BD000000DBB2,BD000000DBET,BD00000037HJ,BD00000037HJ
INSTITUTION STATUS,BANK,BANK,BANK,BANK,BANK
